In [12]:
import os
import re
import torch

def find_matching_files(prefixlist, directory):
    objlist = []
    ptlist = []
    for prefix in prefixlist:
        for filename in os.listdir(directory):
            if filename[:4]== prefix:
                plane = filename[:-4]
                loadobj = plane + '.obj'
                loadpt = plane + '.pt'
                objlist.append(os.path.join(directory, loadobj))
                ptlist.append(os.path.join(directory, loadpt))
                break
    return objlist , ptlist

matching_files = []
in_em1 = [('b827', 'b7fd'), 
          torch.tensor([0, 120]), 
          torch.tensor([180, 120]), 
          torch.tensor([0.0930, 0.0660], dtype=torch.float64)]

objlist , ptlist = find_matching_files(in_em1[0], "./planes")

# 输出匹配的文件
for file in objlist:
    print(file)

for file in ptlist:
    print(file)

print(objlist)

./planes/b82731071bd39b66e4c15ad8a2edd2e.obj
./planes/b7fd11d4af74b4ffddaa0161e9d3dfac.obj
./planes/b82731071bd39b66e4c15ad8a2edd2e.pt
./planes/b7fd11d4af74b4ffddaa0161e9d3dfac.pt
['./planes/b82731071bd39b66e4c15ad8a2edd2e.obj', './planes/b7fd11d4af74b4ffddaa0161e9d3dfac.obj']


In [26]:
import os
import trimesh
import torch
import torch.nn.functional as F

# 示例文件路径列表
file_paths = ['./planes/b82731071bd39b66e4c15ad8a2edd2e.obj', './planes/b7fd11d4af74b4ffddaa0161e9d3dfac.obj']

# 设备（如果有GPU可用则使用CUDA）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 读取并处理文件
def load_and_process(file_path):
    mesh = trimesh.load_mesh(file_path)
    faces = torch.tensor(mesh.faces, dtype=torch.int).unsqueeze(0)
    verts = torch.tensor(mesh.vertices, dtype=torch.float32).unsqueeze(0)
    pt_path = file_path.replace('.obj', '.pt')
    faceedge = torch.load(pt_path)
    return faces, verts, faceedge

# 读取所有文件并合并到batch
faces_list, verts_list, faceedges_list = [], [], []
for file_path in file_paths:
    faces, verts, faceedge = load_and_process(file_path)
    print(f'face={faces.shape},vert={verts.shape},edge={faceedge.shape}')
    faces_list.append(faces)
    verts_list.append(verts)
    faceedges_list.append(faceedge)

# 找到需要填充到的最大大小
max_faces_size = max(f.shape[1] for f in faces_list)
max_verts_size = max(v.shape[1] for v in verts_list)
max_faceedges_size = max(fe.shape[1] for fe in faceedges_list)

# 填充所有张量到相同大小
padded_faces_list = [F.pad(f, (0, 0, 0, max_faces_size - f.shape[1])) for f in faces_list]
padded_verts_list = [F.pad(v, (0, 0, 0, max_verts_size - v.shape[1])) for v in verts_list]
padded_faceedges_list = [F.pad(fe, (0,0,0, max_faceedges_size - fe.shape[1])) for fe in faceedges_list]

# 合并到batch
planesur_faces = torch.cat(padded_faces_list, dim=0).to(device)
planesur_verts = torch.cat(padded_verts_list, dim=0).to(device)
planesur_faceedges = torch.cat(padded_faceedges_list, dim=0).to(device)

# 打印结果以确认
print("Batch of faces tensor shape:", planesur_faces.shape)
print("Batch of vertices tensor shape:", planesur_verts.shape)
print("Batch of face edges tensor shape:", planesur_faceedges.shape)


face=torch.Size([1, 20804, 3]),vert=torch.Size([1, 10400, 3]),edge=torch.Size([1, 83216, 2])
face=torch.Size([1, 12996, 3]),vert=torch.Size([1, 6500, 3]),edge=torch.Size([1, 51984, 2])
Batch of faces tensor shape: torch.Size([2, 20804, 3])
Batch of vertices tensor shape: torch.Size([2, 10400, 3])
Batch of face edges tensor shape: torch.Size([2, 83216, 2])


In [27]:
import os
import trimesh
import torch
import torch.nn.functional as F

def load_and_process(file_path):
    """
    读取并处理单个文件，返回faces, verts, faceedge张量。
    """
    mesh = trimesh.load_mesh(file_path)
    faces = torch.tensor(mesh.faces, dtype=torch.int).unsqueeze(0)
    verts = torch.tensor(mesh.vertices, dtype=torch.float32).unsqueeze(0)
    pt_path = file_path.replace('.obj', '.pt')
    faceedge = torch.load(pt_path)
    return faces, verts, faceedge

def process_files(file_paths, device='cpu'):
    """
    处理一组文件并返回批量张量。
    
    Args:
    - file_paths (list of str): 文件路径列表。
    - device (str or torch.device): 使用的设备，默认为'cpu'。
    
    Returns:
    - planesur_faces (torch.Tensor): faces批量张量。
    - planesur_verts (torch.Tensor): verts批量张量。
    - planesur_faceedges (torch.Tensor): faceedges批量张量。
    """
    # 设置设备
    device = torch.device(device if isinstance(device, str) else device)

    # 读取所有文件并合并到batch
    faces_list, verts_list, faceedges_list = [], [], []
    for file_path in file_paths:
        faces, verts, faceedge = load_and_process(file_path)
        faces_list.append(faces)
        verts_list.append(verts)
        faceedges_list.append(faceedge)

    # 找到需要填充到的最大大小
    max_faces_size = max(f.shape[1] for f in faces_list)
    max_verts_size = max(v.shape[1] for v in verts_list)
    max_faceedges_size = max(fe.shape[1] for fe in faceedges_list)

    # 填充所有张量到相同大小
    padded_faces_list = [F.pad(f, (0, 0, 0, max_faces_size - f.shape[1])) for f in faces_list]
    padded_verts_list = [F.pad(v, (0, 0, 0, max_verts_size - v.shape[1])) for v in verts_list]
    padded_faceedges_list = [F.pad(fe, (0, 0, 0, max_faceedges_size - fe.shape[1])) for fe in faceedges_list]

    # 合并到batch
    planesur_faces = torch.cat(padded_faces_list, dim=0).to(device)
    planesur_verts = torch.cat(padded_verts_list, dim=0).to(device)
    planesur_faceedges = torch.cat(padded_faceedges_list, dim=0).to(device)

    return planesur_faces, planesur_verts, planesur_faceedges

file_paths = ['./planes/b82731071bd39b66e4c15ad8a2edd2e.obj', './planes/b7fd11d4af74b4ffddaa0161e9d3dfac.obj']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
planesur_faces, planesur_verts, planesur_faceedges = process_files(file_paths, device)

# 打印结果以确认
print("Batch of faces tensor shape:", planesur_faces.shape)
print("Batch of vertices tensor shape:", planesur_verts.shape)
print("Batch of face edges tensor shape:", planesur_faceedges.shape)


Batch of faces tensor shape: torch.Size([2, 20804, 3])
Batch of vertices tensor shape: torch.Size([2, 10400, 3])
Batch of face edges tensor shape: torch.Size([2, 83216, 2])
